<a href="https://www.kaggle.com/code/burcakaydn/rating-product-and-sorting-reviews-in-amazon?scriptVersionId=170351470" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Rating Product & Sorting Reviews in Amazon

# Business Problem

* One of the most important problems in e-commerce is the correct calculation of the points given to products after sales.
* The solution to this problem means providing more customer satisfaction for the e-commerce site, making the product stand out for sellers, and a smooth shopping experience for buyers.
* Another problem is the correct ordering of the comments given to the products.
* Highlighting misleading comments will directly affect the sales of the product, causing both financial and customer loss.
* By solving these 2 basic problems, e-commerce sites and sellers will increase their sales, while customers will complete their purchasing journey without any problems.

# Dataset Story


* This data set, which includes Amazon product data, includes product categories and various metadata.
* The product with the most comments in the electronics category has user ratings and comments.

# Variables:
* reviewerID: User ID
* asin: Product ID
* reviewerName: Username
* helpful: Helpful rating rating
* reviewText: Review
* overall: Product rating
* summary: Evaluation summary
* unixReviewTime: Review time
* reviewTime: Review time Raw
* day_diff: Number of days since evaluation
* helpful_yes: Number of times the review was found helpful
* total_vote: Number of votes cast on the review

In [1]:
import pandas as pd
import math
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
df = pd.read_csv("/kaggle/input/amazonreview/amazon_review.csv")
print(df.shape)
df.head(10)

(4915, 12)


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0
5,A2IDCSC6NVONIZ,B007WTAJTO,2Cents!,"[0, 0]",It's mini storage. It doesn't do anything els...,5.00000,Not a lot to really be said,1367193600,2013-04-29,588,0,0
6,A26YHXZD5UFPVQ,B007WTAJTO,2K1Toaster,"[0, 0]",I have it in my phone and it never skips a bea...,5.00000,Works well,1382140800,2013-10-19,415,0,0
7,A3CW0ZLUO5X2B1,B007WTAJTO,"35-year Technology Consumer ""8-tracks to 802.11""","[0, 0]",It's hard to believe how affordable digital ha...,5.00000,32 GB for less than two sawbucks...what's not ...,1404950400,2014-10-07,62,0,0
8,A2CYJO155QP33S,B007WTAJTO,4evryoung,"[1, 1]",Works in a HTC Rezound. Was running short of ...,5.00000,Loads of room,1395619200,2014-03-24,259,1,1
9,A2S7XG3ZC4VGOQ,B007WTAJTO,53rdcard,"[0, 0]","in my galaxy s4, super fast card, and am total...",5.00000,works great,1381449600,2013-11-10,393,0,0


In [4]:
df.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime', 'day_diff',
       'helpful_yes', 'total_vote'],
      dtype='object')

## TASK 1:
### Calculate the Average Rating Based on Current Comments and Compare it with the Existing Average Rating.

* In the share# Step 1: Read the Data Set and Calculate the Average Score of the Product.

# Calculate average scored data set, users rated a product and made comments.
* Our aim in this task is to evaluate the given scores by weighting them according to date.
* It is necessary to compare the initial average score with the weighted score according to the date to be obtained.

In [5]:
# Step 1: Read the Data Set and Calculate the Average Score of the Product.

# Calculate mean score
average_rating = df['overall'].mean()
print(f"Mean score: {average_rating}")


Mean score: 4.587589013224822


In [6]:
# Step 2: Calculate the Weighted Score Average by Date.

# Convert reviewTime column to date format
df['reviewTime'] = pd.to_datetime(df['reviewTime'])

# Determine the most recent comment date
current_date = df['reviewTime'].max()


In [7]:

# Calculate the day difference for each comment
df['days_since_review'] = (current_date - df['reviewTime']).dt.days

# Calculating weights and calculating weighted average score
def time_based_weighted_average(dataframe, w1=50, w2=25, w3=10, w4=10):
    return dataframe.loc[df["days_since_review"] <= 30, "overall"].mean() * w1 / 100 + \
           dataframe.loc[(dataframe["days_since_review"] > 30) & (dataframe["days_since_review"] <= 90), "overall"].mean() * w2 / 100 + \
           dataframe.loc[(dataframe["days_since_review"] > 90) & (dataframe["days_since_review"] <= 180), "overall"].mean() * w3 / 100 + \
           dataframe.loc[(dataframe["days_since_review"] > 180), "overall"].mean() * w4 / 100

time_based_weighted_average(df)

4.503678531512139

## Task 2:
### Determine 20 Reviews to Be Displayed on the Product Detail Page for the Product.


In [8]:
# Step 1. Generate the variable helpful_no
df['helpful_no'] = df['total_vote'] - df['helpful_yes']

In [9]:
# Step 2. Calculate score_pos_neg_diff, score_average_rating and wilson_lower_bound Scores and Add them to the Data

def score_up_down_diff(helpful_yes, helpful_no):
    return helpful_yes - helpful_no

df["score_pos_neg_diff"] = df.apply(lambda x: score_up_down_diff(x["helpful_yes"], 
                                                                 x["helpful_no"]), axis=1)
df.loc[df["score_pos_neg_diff"] > 2, ["reviewerID", "overall",  
                                      "helpful_yes", "total_vote", "days_since_review",
                                      "score_pos_neg_diff"]].sample(5)

,reviewerID,overall,helpful_yes,total_vote,days_since_review,score_pos_neg_diff
1072,A2O96COBMVY9C4,5.00000,5,5,941,5
1609,A2TPXOZSU1DACQ,5.00000,7,7,256,7
1835,A1J6VSUM80UAF8,5.00000,60,68,282,52
4883,A1YEPFLLH42OU1,4.00000,6,8,738,4
323,A15X60NOGL3WDW,5.00000,6,7,337,5


In [10]:
from scipy.stats import norm
import numpy as np

In [11]:
def wilson_lower_bound(up, down, confidence=0.95):
    """
    Function calculating Wilson Lower Bound score.
    """
    n = up + down
    if n == 0:
        return 0
    z = norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return ((phat + z**2 / (2*n) - z * np.sqrt((phat*(1-phat)+z**2/(4*n))/n))/(1+z**2/n))

# Calculate Wilson Lower Bound score and insert it to the column
df['wilson_lower_bound'] = df.apply(lambda x: wilson_lower_bound(x['helpful_yes'], x['helpful_no']), axis=1)

In [12]:
# Step 3. Identify 20 Comments and Interpret the Results.

# Select the top 20 comments based on Wilson Lower Bound score
top_20_reviews = df.sort_values(by='wilson_lower_bound', ascending=False).head(20)

# Show summary of selected comments
print(top_20_reviews[['reviewText', 'wilson_lower_bound']])

                                             reviewText  wilson_lower_bound
2031  [[ UPDATE - 6/19/2014 ]]So my lovely wife boug...             0.95754
3449  I have tested dozens of SDHC and micro-SDHC ca...             0.93652
4212  NOTE:  please read the last update (scroll to ...             0.91214
317   If your card gets hot enough to be painful, it...             0.81858
4672  Sandisk announcement of the first 128GB micro ...             0.80811
1835  Bought from BestBuy online the day it was anno...             0.78465
3981  The last few days I have been diligently shopp...             0.73214
3807  I bought this card to replace a lost 16 gig in...             0.70044
4306  While I got this card as a "deal of the day" o...             0.67033
4596  Hi:I ordered two card and they arrived the nex...             0.66359
315   Bought this card to use with my Samsung Galaxy...             0.65741
1465  I for one have not bought into Google's, or an...             0.64567
1609  I have

In [13]:
# Step 3. Identify 20 Comments and Interpret the Results.

# Select the top 20 comments based on Wilson Lower Bound score
top_50_reviews = df.sort_values(by='wilson_lower_bound', ascending=False).head(50)

# Show summary of selected comments
print(top_50_reviews[['reviewText', 'wilson_lower_bound']])

                                             reviewText  wilson_lower_bound
2031  [[ UPDATE - 6/19/2014 ]]So my lovely wife boug...             0.95754
3449  I have tested dozens of SDHC and micro-SDHC ca...             0.93652
4212  NOTE:  please read the last update (scroll to ...             0.91214
317   If your card gets hot enough to be painful, it...             0.81858
4672  Sandisk announcement of the first 128GB micro ...             0.80811
1835  Bought from BestBuy online the day it was anno...             0.78465
3981  The last few days I have been diligently shopp...             0.73214
3807  I bought this card to replace a lost 16 gig in...             0.70044
4306  While I got this card as a "deal of the day" o...             0.67033
4596  Hi:I ordered two card and they arrived the nex...             0.66359
315   Bought this card to use with my Samsung Galaxy...             0.65741
1465  I for one have not bought into Google's, or an...             0.64567
1609  I have